To begin, we import all the relevant modules.

In [ ]:
from datetime import datetime
from tkinter import Tk

from aer.experimentalist.experiment_design_synthetic_weber import (
    Experiment_Design_Synthetic_Weber,
)
from aer.experimentalist.experimentalist_popper import Experimentalist_Popper
from aer.theorist.theorist_darts import DARTS_Type, Theorist_DARTS
from aer.theorist.theorist_GUI import Theorist_GUI
from example.weber.weber_setup import general_parameters, study_object, validation_object_1

We then note the starting time.

In [ ]:
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

From here, we specify our experiment design and initialize our preferred experimentalist.

Specify our experiment design:

In [ ]:
stimulus_resolution = 20
weber_design = Experiment_Design_Synthetic_Weber(stimulus_resolution)

stimulus_resolution_validation = 100
weber_design_validation = Experiment_Design_Synthetic_Weber(
    stimulus_resolution_validation
)

Initialize the experimentalist:

In [ ]:
experimentalist = Experimentalist_Popper(
    study_name=general_parameters.study_name,
    experiment_server_host=general_parameters.host,
    experiment_server_port=general_parameters.port,
    experiment_design=weber_design,
)

experimentalist_validation = Experimentalist_Popper(
    study_name=general_parameters.study_name_sampled,
    experiment_server_host=general_parameters.host,
    experiment_server_port=general_parameters.port,
    experiment_design=weber_design_validation,
)

We also need to initialize our preferred theorist

In [ ]:
theorist = Theorist_DARTS(general_parameters.study_name, darts_type=DARTS_Type.ORIGINAL)

and specify plots for it.

In [ ]:
plots = list()
plots.append(theorist._loss_plot_name)
theorist.plot()

At this point we can execute Autonomous Empirical Research.

First, we generate our initial validation data set.

In [ ]:
# validation_data = experimentalist_validation.seed(validation_object_1) # seed with new experiment
validation_data = experimentalist_validation.seed(
    validation_object_1, datafile="experiment_0_data.csv"
)  # seed with new experiment
validation_object_1.add_data(validation_data)

Then, we seed the experimentalist with the data we just generated and split it into training and validations sets.

In [ ]:
# Seed experiment and split into training/validation set

# seed_data = experimentalist.seed(study_object) # seed with new experiment
seed_data = experimentalist.seed(
    study_object, datafile="experiment_0_data.csv"
)  # seed with existing data file
study_object.add_data(seed_data)
validation_object_2 = study_object.split(proportion=0.5)
validation_object_2.name = "Weber Sampled"

At this point, we can provide the theorist with the same two data sets.

In [ ]:
theorist.add_validation_set(validation_object_1, "Weber_Sampled")
theorist.add_validation_set(validation_object_2, "Weber_Original")

With the initializations and seeding complete, the theorist can perform its search for the best model.

In [ ]:
# model = theorist.search_model(study_object)

root = Tk()
app = Theorist_GUI(object_of_study=study_object, theorist=theorist, root=root)
root.mainloop()

# root = Tk()
# app = AER_GUI(object_of_study=study_object, theorist=theorist,
# experimentalist=experimentalist, root=root)
# root.mainloop()

# theorist_fair = Theorist_DARTS(study_name, darts_type=DARTS_Type.FAIR)
# theorist_fair.plot()
# theorist_fair.add_validation_set(validation_object_1, 'Weber_Sampled')
# theorist_fair.add_validation_set(validation_object_2, 'Weber_Original')
# model = theorist_fair.search_model(study_object)

Finally, we make note of the time after model search has been conducted.

In [ ]:
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
